# MNIST 手写数字识别

首先，导入 keras 包，导入 mnist 数据集

In [2]:
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images 和 train_labels 组成了训练集，模型将从这些数据中进行学习。然后再测试集 test_images 和 test_labels 上对模型进行测试。

图像被编码为 Numpy 数组，而标签是数字数组，取值范围为 0~9。图像和标签一一对应。

In [3]:
train_images.shape

(60000, 28, 28)

In [4]:
len(train_labels)

60000

In [5]:
train_labels

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [7]:
test_images.shape

(10000, 28, 28)

下面是测试数据：

In [8]:
len(test_labels)

10000

In [6]:
test_images.shape

(10000, 28, 28)

In [7]:
test_labels

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

接下来的工作流程：

- 将训练数据（train_images 和 train_labels）输入神经网络；
- 网络学习将图像和标签关联在一起；
- 使用网络对 test_images 生成预测，验证这些预测与 test_labels 中的标签是否匹配。

下面构建网络

In [8]:
from tensorflow.keras import layers
from tensorflow.keras import models

In [9]:
network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))

神经网络的核心是层（layer），它是一种数据处理模块。具体来说，层从输入数据种提取表示——我们期望这种表示有助于解决手头的问题。大多数深度学习都是将简单的层链接起来，从而实现渐进式的**数据蒸馏**（data distillation）。深度学习模型就像是一个数据处理的筛子，包含一系列越来越精细的数据过滤器。

上面的网络包含 2 个 Dense 层，它们是密集连接（也叫全连接）的神经层。第二层是一个 10 路 softmax 层，它将返回一个由 10 个概率值（总和为 1）组成的数组。每个概率值表示当前数字图像属于 10 个数字类别中某一个的概率。

要向训练网络，还需要进行编译步骤的三个参数：

- 损失函数（loss function）：衡量在训练数据上的性能，即网络如何朝着正确的方向前进。
- 优化器（optimizer）：基于训练数据和损失函数来更新网络的机制。
- 在训练和测试过程中需要监控的指标：本例只关心精度，即正确分类的图像所占的比例。

In [10]:
network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

在训练之前，我们需要将数据变换为网络要求的形状，并缩放到所有值在 [0,1] 区间。比如，之前训练图像保存在 uint8 类型的数组种，其形状为 (60000, 28, 28)，取值区间为 [0, 255]。我们需要将其变换为 float32 数组，形状为 (60000, 28*28)，取值范围 0~1

In [11]:
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

接下来对标签进行分类编码。

In [12]:
from tensorflow.keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

准备开始训练网络，在 keras 中这一步是通过调用网络的 `fit` 方法来完成的

In [13]:
network.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
469/469 [==============================] - 3s 5ms/step - loss: 0.2561 - accuracy: 0.9262
Epoch 2/5
469/469 [==============================] - 2s 5ms/step - loss: 0.1045 - accuracy: 0.9694
Epoch 3/5
469/469 [==============================] - 2s 5ms/step - loss: 0.0686 - accuracy: 0.9792
Epoch 4/5
469/469 [==============================] - 2s 5ms/step - loss: 0.0500 - accuracy: 0.9849
Epoch 5/5
469/469 [==============================] - 2s 5ms/step - loss: 0.0378 - accuracy: 0.9886


上面显示两个数字：

- 训练数据上的损失（loss）；
- 网络在训练数据上的精度（accuracy）；

上面很快就在训练数据上达到了 98.86%的精度。下面我们检查模型在测试集上的性能

In [14]:
test_loss, test_acc = network.evaluate(test_images, test_labels)

313/313 [==============================] - 0s 651us/step - loss: 0.0739 - accuracy: 0.9783


测试精度为97.83 %，比训练精度低。训练精度和测试精度之间的这种差距就是**过拟合**（overfit）。过拟合是指机器学习模型在新数据上的性能比在训练数据上要差。